In [11]:
import cv2 
import numpy as np
import os
def get_txt_files(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]
def get_image_files(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]

def rotate_yolo_labels_90(yolo_labels_path, img_width, img_height, num_rotations=1, prev_rotation=None) -> list:
    if prev_rotation == None:
        with open(yolo_labels_path, "r") as f:
            lines = f.readlines()
    else:
        lines = prev_rotation

    rotated_lines = []

    for line in lines:
        class_idx, x_center_norm, y_center_norm, width_norm, height_norm = map(float, line.split())
        class_idx = int(class_idx)

        # Convert normalized center coordinates to pixel coordinates
        x_center = x_center_norm * img_width
        y_center = y_center_norm * img_height

        # Compute new pixel coordinates for rotated bounding box
        x_new = y_center
        y_new = img_width - x_center

        # Convert new pixel coordinates back to normalized coordinates
        x_new_norm = x_new / img_height
        y_new_norm = y_new / img_width

        # NEW ADDED
        width_new_norm = height_norm
        height_new_norm = width_norm

        # Append rotated annotation to list of strings
        rotated_line = f"{class_idx} {x_new_norm:.6f} {y_new_norm:.6f} {width_new_norm:.6f} {height_new_norm:.6f}"
        rotated_lines.append(rotated_line)
    
    if num_rotations == 1:
        return rotated_lines
    else:
        rotated_lines = rotate_yolo_labels_90(yolo_labels_path, width_new_norm, height_new_norm, num_rotations-1, rotated_lines)

    return rotated_lines
def rotated_true_labels(folder_path, img_width, img_height, num_rotations) -> str:
    output_list = rotate_yolo_labels_90(folder_path, img_width, img_height, num_rotations)
    #now append the list together as a string separated by newlines
    return "\n".join(output_list)



In [12]:
def preprocess_images(img_folder_path):
    #get every image in the path
    image_files = get_image_files(img_folder_path)
    for image_file in image_files:
        #get the image_file name without extension
        base_name = os.path.splitext(os.path.basename(image_file))[0]
        image = cv2.imread(image_file)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # normalized_image = gray_image / 255.0
        blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

        #now time for some data augmenting
            #let's add rotated version of the image to the dir as 0 thru 3 
        rotated_image = blurred_image
        augmented_images = [blurred_image]
        for i in range(3):
            rotated_image = cv2.rotate(rotated_image, cv2.ROTATE_90_CLOCKWISE)
            augmented_images.append(rotated_image)

        #finally, write the image to preprocessed/images/base_name_0 thru 3 .jpg
        for i, augmented_image in enumerate(augmented_images):
            cv2.imwrite(f'preprocessed/images/{base_name}_{i}.jpg', augmented_image)

        #now, write the rotated image lables to preprocessed/labels/base_name_0 thru 3 .txt


In [24]:
def preprocess_labels(label_folder_path):
    label_files = get_txt_files(label_folder_path)
    for label_file in label_files:
        base_name = os.path.splitext(os.path.basename(label_file))[0] 
        for i in range(4):
                with open(f'preprocessed/labels/{base_name}_{i}.txt', 'w') as f:
                    if i == 0:
                        f.write(open(label_file).read())
                    else:
                        f.write(rotated_true_labels(label_folder_path+"/"+base_name+'.txt', 640, 640, i))
    

In [15]:
path = "Training Set/Training Images/"
preprocess_images(path)

KeyboardInterrupt: 

In [25]:
path = 'Training Set/Training Coordinates'
preprocess_labels(path)